# HowTo -TriangleMesh- From point cloud to triangular mesh

In this notebook you will learn:

1. how to connect to a ROMI database
2. how to transform the carved volume into point cloud
3. how to visualize the reconstructed point cloud

This notebook **assume** that you have:
- declared the `ROMI_DB` environment variable as the path to the database directory to use
- processed the test dataset with the _geometric pipeline,_ so we can access the fileset containing the data we want to start with...

Remember, the aim of this notebook is to show you how it works "under the hood".
This is not how you should process your data, that is done thanks to the `romi_run_task` CLI tool.

In [ ]:
import os

import ipywidgets as widgets
import open3d as o3d
import plotly.graph_objects as go
from plant3dvision.visu import plotly_pointcloud, plotly_mesh, plotly_pointcloud_data, plotly_mesh_data
from plantdb import FSDB
from plantdb.io import read_point_cloud

## Connect to the database & get the initial data

If you did not declare a `ROMI_DB` environment variable, you can do it by uncommenting the next cell and setting it to the right value.

In [ ]:
# os.environ['ROMI_DB'] = "/path/to/test/data"

### Connect to the database

In [ ]:
db = FSDB(os.environ['ROMI_DB'])  # requires definition of this environment variable!
db.connect()

Once you are connected to the database, you can list the available scan *dataset* with `db.list_scans()`.

### Select a dataset

We now select a dataset (with the `Dropdown` widget) for the demo:

In [ ]:
scan_name = widgets.Dropdown(options=db.list_scans(), value=db.list_scans()[0], description='Dataset:')
display(scan_name)

In [ ]:
scan = db.get_scan(scan_name.value)

If you did not process this dataset yet, from the `plant3dvision` root directory, you can do it with:
```
romi_run_task AnglesAndInternodes $ROMI_DB/<selected_dataset> --config plant-3d-vision/configs/geom_pipe_real.toml
```

To list the available *filesets* in this *scan dataset*:

In [ ]:
scan.list_filesets()

### Get the point cloud fileset

The point cloud resulting from a _PointCloud_ task is to be found in the 'PointCloud*' fileset.

In [ ]:
pcd_fs = scan.get_filesets(query={"task_name": "PointCloud"})[0]
print(pcd_fs.path().stem)

### Get and load the point cloud file

Once you have access to the right fileset, you may access the point cloud file and load it as follows:

In [ ]:
pcd_file = pcd_fs.get_files()[0]
pcd = read_point_cloud(pcd_file)

### Visualize the point cloud

It is possible to visualize the point cloud with the `plotly_pointcloud` method from `plant3dvision.visu` as follows:

In [ ]:
fig = plotly_pointcloud(pcd, n_pts=9000)

In [ ]:
fig.show()

## Computing the triangular mesh

It is now time to process the *point cloud* to generate a *triangular mesh*.

### Processing with Open3d library

This is the default library used to create triangles mesh from point cloud in the geometry-based workflow.

In [ ]:
tmesh = o3d.geometry.TriangleMesh()
tmesh, _ = tmesh.create_from_point_cloud_poisson(pcd, depth=9)

### Processing with CGAL library

For the sake of completeness we also introduce how it can be done with CGAL.

In [ ]:
# tmesh = pcd2mesh(pcd)

### Visualize the triangular mesh

Once the point cloud has been obtained, we can visualize it with the `plotly_mesh` method from `plant3dvision.visu` as follows:

In [ ]:
fig = plotly_mesh(tmesh)

In [ ]:
fig.show()

## Compare the point cloud and mesh

Let's represent both objects and see how close they match:

Now we generate graph objects for point cloud and mesh:

In [ ]:
go_pcd = plotly_pointcloud_data(pcd, marker_kwargs={'size': 3})
go_mesh = plotly_mesh_data(tmesh, mesh_kwargs={'opacity': 0.6})

Finally, we use Plotly to represent all this:

In [ ]:
layout_style = dict(height=800, width=800, showlegend=False)
fig = go.Figure(data=[go_pcd, go_mesh], layout=layout_style)
fig.update_scenes(aspectmode='data')

We may now **disconnect** from the database as we will not need it anymore:

In [ ]:
db.disconnect()